

신예린님 깃허브 참고함.
https://github.com/rinrin528/AIFFEL-Exploration/blob/main/%5BE-18%5D%20Compare%20OCR%20Libraries.ipynb

# OCR 라이브러리 비교하기



In [ ]:
import os
import glob

import cv2

import matplotlib.pyplot as plt
import matplotlib.image as img

# keras-ocr
import keras_ocr

# tesseract
import pytesseract
from PIL import Image
from pytesseract import Output
import re

# Google OCR API
from google.cloud import vision
import io
from PIL import ImageDraw

# Step1. 검증용 데이터셋 준비

In [ ]:
data_path = os.getenv('HOME')+'/aiffel/enode18/ocr_python/tea/'

In [ ]:
images_path = glob.glob(data_path+"*.jpg")
images_path

In [ ]:
plt.figure(figsize=(15, 15))

for i, image in enumerate(images_path,1):
    plt.subplot(5, 3, i)
    plt.imshow(img.imread(image))
    plt.axis('off')

# Step2. Google OCR API, keras-ocr, Tesseract로 테스트 진행

In [ ]:
# keras-ocr이 detector과 recognizer를 위한 모델을 자동으로 다운로드받게 됩니다. 
pipeline = keras_ocr.pipeline.Pipeline()

In [ ]:
images = [keras_ocr.tools.read(im) for im in images_path]
predictions = [pipeline.recognize([im]) for im in images_path]

In [ ]:
fig, axs = plt.subplots(5, 3, figsize=(30, 30))
for a in range(len(axs)):
    for b in range(len(axs[a])):
        keras_ocr.tools.drawAnnotations(image=images[3*a+b], 
                                    predictions=predictions[3*a+b][0], ax=axs[a,b])

In [ ]:
def detect_box(image_path):
    filename = os.path.basename(image_path)
    # read the image and get the dimensions
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    d = pytesseract.image_to_data(img, output_type=Output.DICT)
    n_boxes = len(d['text'])

    # draw the bounding boxes on the image
    for i in range(n_boxes):
        if int(d['conf'][i]) > 1: # confidence가 1 이상인 값들만
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            text = d['text'][i]
            text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
            cv2.putText(img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        1.5, (0, 0, 255), 3)
    return img

In [ ]:
# confidence = 1
plt.figure(figsize=(15, 15))
for i,image_path in enumerate(images_path,1):
    img = detect_box(image_path)
    plt.subplot(5, 3, i)
    plt.imshow(img)
    plt.axis('off')

In [ ]:
def extract_text(image_path):
    custom_oem_psm_config = r'--oem 3 --psm 11'
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    txt = pytesseract.image_to_string(img, lang='eng',config=custom_oem_psm_config)
    txt = re.sub(r"\n+"," ",txt)
    return txt

In [ ]:
extract_text(images_path[1])

In [ ]:
for image_path in images_path:
    filename = os.path.basename(image_path)
    txt = extract_text(image_path)
    txt = "".join([c if ord(c) < 128 else "" for c in txt]).strip()
    print(f'========{filename}========')
    print(txt)

In [ ]:
def detect_text(path):
    """Detects text in the file."""
    text_dec = []
    bounds = []
    client = vision.ImageAnnotatorClient()
    
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    
    name = os.path.basename(path)
    
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    for text in texts:
        text_dec.append(text.description.replace('\n',''))
        vertices = text.bounding_poly
        bounds.append(vertices)
    
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
        
    return name, text_dec, bounds

In [ ]:
def draw_boxes(image, bounds, color):
    """Draw a border around the image using the hints in the vector list."""
    draw = ImageDraw.Draw(image)

    for bound in bounds:
        draw.polygon([
            bound.vertices[0].x, bound.vertices[0].y,
            bound.vertices[1].x, bound.vertices[1].y,
            bound.vertices[2].x, bound.vertices[2].y,
            bound.vertices[3].x, bound.vertices[3].y], None, color)
    return image

In [ ]:
!ls -l $GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] =  os.getenv('HOME')+'/aiffel/enode18/ocr_python/hopeful-flash-326517-995a4ddfc997.json'

In [ ]:
for path in images_path:
    detect_text(path)

In [ ]:
ocr_img = []
ocr_dict = {}
# 위에서 정의한 OCR API 이용 함수를 호출해 봅시다.
for path in images_path:
    name, text_dec, bounds = detect_text(path)
    ocr_dict[name] = text_dec
    image = Image.open(path)
    draw_boxes(image, bounds, 'red')
    ocr_img.append(image)

In [ ]:
plt.figure(figsize=(15, 15))

for i, img in enumerate(ocr_img, 1):
    plt.subplot(5, 3, i)
    plt.imshow(img)
    plt.axis('off')

# Step3. 테스트 결과 정리

In [ ]:
def show_keras(num):
    keras_ocr.tools.drawAnnotations(image=images[num], predictions=predictions[num][0])

In [ ]:
show_keras(3)

In [ ]:
show_keras(10)

In [ ]:
show_keras(2)

In [ ]:
show_keras(4)

In [ ]:
show_keras(8)

In [ ]:
def show_tesseract(num):
    plt.figure(figsize=(15, 15))

    plt.subplot(121)
    org = cv2.imread(images_path[num])
    org = cv2.cvtColor(org, cv2.COLOR_BGR2RGB)
    plt.imshow(org)
    plt.title('Original')
    plt.subplot(122)
    img = detect_box(images_path[num])
    plt.imshow(img)
    plt.title('Detected')
    txt = extract_text(images_path[num])
    print(txt)

In [ ]:
show_tesseract(8)

In [ ]:
show_tesseract(-1)

In [ ]:
show_tesseract(11)

In [ ]:
show_tesseract(4)

In [ ]:
def show_google(num):
    name, text_dec, bounds = detect_text(images_path[num])
    image = Image.open(images_path[num])
    
    plt.figure(figsize=(15, 15))
    plt.subplot(121)
    org = image
    plt.imshow(org)
    plt.title('Original')
    plt.subplot(122)
    img = draw_boxes(image, bounds, 'red')
    plt.imshow(img)
    plt.title('Detected')
    print(text_dec[0])

In [ ]:
show_google(4)

In [ ]:
show_google(11)

# Step4. 결과 분석과 결론 제시

- 차 케이스 사진으로 제품 정보를 검색해주는 서비스를 위해서는 제품명과 세부 맛에 관련된 텍스트를 정확하게 검출해야한다.
- 특수문자 부분은 전처리해서 지울 수 있으니 최대한 많은 텍스트를 인식하는지 여부를 보자. ### 분석 기준
1. 브랜드명을 정확히 검출했는지
2. 핵심 맛 정보를 정확히 검출했는지
3. 티백 개수, 무게 등 상세 정보를 검출했는지

이미지	keras-ocr	tesseract	Google OCR API
0	o/o/o	x/x/x	o/o/o
1	x/o/o	x/x/x	o/o/o
2	o/o/o	x/x/o	o/o/o
3	o/x/o	x/o/x	o/o/o
4	x/o/x	x/o/x	o/o/o
5	o/o/o	x/x/x	o/o/o
6	o/o/x	x/o/x	o/o/o
7	x/o/x	x/x/x	o/o/o
8	o/o/o	o/x/x	o/o/o
9	o/o/o	x/x/x	o/o/o
10	o/o/o	x/x/x	o/o/o
11	o/o/o	x/o/x	o/o/o
12	o/o/o	x/o/x	o/o/o
13	o/x/x	x/x/x	o/o/o
14	o/o/o	o/o/o	o/o/o

- Google OCR API의 결과가 월등히 우수했다.
- tesseract는 사용하기 어려운 수준의 정확도를 보였다.
- 비용이 걱정된다면 keras-ocr로 적당히 타협을 보는 것도 좋은 대안일 것이다.

- 서비스가 고도의 정확도를 요구하다면 Google OCR API를 사용할 것이고, 비용 절감이 더 우선인 경우 keras-ocr을 사용하는 것이 적합해 보인다.

# 회고

- 분석을 되게 치밀하게 잘 하셔서 보는 내내 놀랬다.
- 하지만 여전히 나는 이해를 못했다.
- 파이썬에 대한 개념이 정확하게 안잡혀서 그런가, 알고리즘에 대한 내용이 안잡혀서 그런가 너무나도 먼 산일뿐.
- 별사탕 프로젝트로 회생하기를 바랄뿐이다.